In [2]:
from google.colab import files

In [3]:
import pandas as pd
import numpy as np
import torch.nn as nn

In [4]:
upl = files.upload()

Saving train (1).csv to train (1).csv
Saving test (1).csv to test (1).csv


In [5]:
train = pd.read_csv('train (1).csv')
test = pd.read_csv('test (1).csv')

In [6]:
train

,id,phrase,feature_1,feature_2,feature_3,sentiment
0,0,It may as well be called `` Jar-Jar Binks : Th...,14.0,5.0,7.0,0
1,1,You have to see it .,6.0,1.0,NaN,2
2,2,... either you 're willing to go with this cla...,16.0,0.0,6.0,1
3,3,Watching Harris ham it up while physically and...,37.0,NaN,3.0,1
4,4,Pete 's screenplay manages to find that real n...,20.0,1.0,4.0,2
...,...,...,...,...,...,...
6995,6995,"A fantastic premise anchors this movie , but w...",41.0,3.0,NaN,1
6996,6996,"A boring , pretentious muddle that uses a sens...",37.0,2.0,11.0,0
6997,6997,"Boy , has this franchise ever run out of gas .",11.0,1.0,2.0,0
6998,6998,It 's hard to understand why anyone in his rig...,21.0,1.0,2.0,0


In [7]:
train['phrase'].iloc[0]

"It may as well be called `` Jar-Jar Binks : The Movie . ''"

In [8]:
D = {}

In [9]:
D['Unknown'] = 0

In [13]:
sample_data = train.iloc[:10]['phrase'].to_list()
sents = train.iloc[:10]['sentiment']

In [14]:
sents

,sentiment
0,0
1,2
2,1
3,1
4,2
5,0
6,0
7,2
8,2
9,1


In [10]:
def create_dict(x):
  r = x.split(" ")
  for w in r:
    if w.isalpha() and w.lower() not in D:
      D[w.lower()] = len(D)


In [11]:
train['phrase'].apply(lambda x:create_dict(x))

,phrase
0,None
1,None
2,None
3,None
4,None
...,...
6995,None
6996,None
6997,None
6998,None


In [12]:
len(D)

13141

In [24]:
embd_dim = 8
embedd_mat = np.random.randn(len(D),embd_dim)*0.01

In [13]:
def convert_integer(x):
  r = x.split(" ")
  l = []
  for w in r:
    if w.isalpha() and w.lower() in D:
      l.append(D[w.lower()])
    else:
      l.append(0)


  return l

In [14]:
convert_integer("Lagaan Bollywood")

[90, 93]

In [42]:
sequences = [convert_integer(r) for r in sample_data]

In [45]:
def convert_embbeds(x):
  return np.array([embedd_mat[i] for i in x])

In [130]:
hidden_dim = 16
Wxh = np.random.randn(embd_dim,hidden_dim)*0.01
Whh = np.random.randn(hidden_dim,hidden_dim)*0.01
bh = np.random.randn(hidden_dim)
Why = np.random.randn(hidden_dim,3)*0.01
by = np.random.randn(3)

In [131]:
def hidden_states(embbeds):
  states = []
  h_prev = np.zeros(hidden_dim)
  for emb in embbeds:
    h_prev = np.tanh(emb@Wxh + h_prev@Whh + bh)
    states.append(h_prev)
  return states

In [132]:
def softmax(x):
  e = np.exp(x-np.max(x))
  return e/np.sum(e)

In [133]:
def output(h):
  logits = h@Why + by
  return softmax(logits)

In [134]:
def cross_entropy(pred, target):
    return -np.log(pred[target] + 1e-9)


In [135]:
def forward(X,label):
  embbeds = convert_embbeds(X)
  hs = hidden_states(embbeds)
  h_last = hs[-1]
  pred = output(h_last)
  loss = cross_entropy(pred, label)
  return pred,hs,embbeds ,loss

In [136]:
def backprop(probs, label, hidden, embeds):
    dWxh = np.zeros_like(Wxh)
    dWhh = np.zeros_like(Whh)
    dWhy = np.zeros_like(Why)
    dbh  = np.zeros_like(bh)
    dby  = np.zeros_like(by)

    y_grad = probs.copy()
    y_grad[label] -= 1

    dWhy += np.outer(hidden[-1], y_grad)
    dby  += y_grad

    dh = y_grad @ Why.T
    for t in reversed(range(len(hidden))):
        h = hidden[t]
        dz = (1 - h**2) * dh

        dbh += dz
        dWxh += np.outer(embeds[t], dz)

        h_prev = hidden[t-1] if t > 0 else np.zeros(hidden_dim)
        dWhh += np.outer(h_prev, dz)


        dh = dz @ Whh.T

    return dWxh, dWhh, dWhy, dbh, dby


In [137]:
epoch = 2000
lr = 0.005

In [138]:
for i in range(epoch):
  total_loss = 0
  for label,encode in zip(sents,sequences):

    pred,hs,embbeds ,loss = forward(encode,label)

    dWxh, dWhh, dWhy, dbh, dby = backprop(pred,label,hs,embbeds)
    Wxh -= lr*dWxh
    Whh -= lr*dWhh
    Why -= lr*dWhy
    bh -= lr*dbh
    by -= lr*dby
    total_loss += loss
  print(f'loss for all sentences in {i}th epoch is {total_loss}')



loss for all sentences in 0th epoch is 16.79679719677856
loss for all sentences in 1th epoch is 15.854205602253124
loss for all sentences in 2th epoch is 15.024209341604394
loss for all sentences in 3th epoch is 14.303899444123607
loss for all sentences in 4th epoch is 13.688066241155664
loss for all sentences in 5th epoch is 13.169259840253844
loss for all sentences in 6th epoch is 12.738237638067938
loss for all sentences in 7th epoch is 12.384654007350054
loss for all sentences in 8th epoch is 12.097811099487458
loss for all sentences in 9th epoch is 11.867318437395967
loss for all sentences in 10th epoch is 11.683574528448647
loss for all sentences in 11th epoch is 11.53805071140357
loss for all sentences in 12th epoch is 11.42340347110764
loss for all sentences in 13th epoch is 11.333461766364785
loss for all sentences in 14th epoch is 11.263136941597585
loss for all sentences in 15th epoch is 11.20829382194729
loss for all sentences in 16th epoch is 11.165609879360606
loss for al

In [95]:
sample_test_Data = train.iloc[10:30]['phrase'].to_list()

In [96]:
sample_test_Data

["Whenever you think you 've figured out Late Marriage , it throws you for a loop .",
 'Well-done supernatural thriller with keen insights into parapsychological phenomena and the soulful nuances of the grieving process .',
 'A gratingly unfunny groaner littered with zero-dimensional , unlikable characters and hackneyed , threadbare comic setups .',
 "I could n't recommend this film more .",
 'It remains to be seen whether Statham can move beyond the crime-land action genre , but then again , who says he has to ?',
 "Take away the controversy , and it 's not much more watchable than a Mexican soap opera .",
 'But the second half of the movie really goes downhill .',
 'There are plenty of scenes in Frida that do work , but rarely do they involve the title character herself .',
 "You can taste it , but there 's no fizz .",
 "A movie that 's held captive by mediocrity .",
 "Just because A Walk to Remember is shrewd enough to activate girlish tear ducts does n't mean it 's good enough for 

In [127]:
test_sequences = [convert_integer(r) for r in sample_test_Data]

In [128]:
preds = []
for s in test_sequences:
  pred,_,_,_ = forward(s,0)
  preds.append(pred)

In [129]:
np.argmax(preds,axis=1)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [82]:
train.iloc[10:16]

,id,phrase,feature_1,feature_2,feature_3,sentiment
10,10,Whenever you think you 've figured out Late Ma...,17.0,3.0,3.0,2
11,11,Well-done supernatural thriller with keen insi...,18.0,1.0,2.0,2
12,12,A gratingly unfunny groaner littered with zero...,17.0,1.0,4.0,0
13,13,I could n't recommend this film more .,8.0,1.0,NaN,2
14,14,It remains to be seen whether Statham can move...,25.0,2.0,4.0,1
15,15,"Take away the controversy , and it 's not much...",18.0,2.0,3.0,0


In [15]:
from torch.utils.data import Dataset,DataLoader
import torch

In [16]:
class customdata(Dataset):
  def __init__(self,X,y):
    seq = [convert_integer(t) for t in X]
    mx = max(len(s) for s in seq)
    pad = [ s + [0]*(mx-len(s)) for s in seq]
    self.sequences  = torch.tensor(pad,dtype=torch.long)
    self.label = torch.tensor(y,dtype=torch.long)
  def __len__(self):
    return len(self.sequences)
  def __getitem__(self,index):
    return self.sequences[index],self.label[index]


In [17]:
X = train['phrase'].to_list()
y = train['sentiment']

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2)

In [197]:
se = torch.tensor([convert_integer(r) for r in X_train])

ValueError: expected sequence of length 8 at dim 1 (got 14)

In [20]:
train_set = customdata(X_train,Y_train.to_numpy())
test_set = customdata(X_test,Y_test.to_numpy())

In [21]:
train_set.__getitem__(3)

(tensor([  18, 8756,  506,  518, 1548,  803,    0,   29,    1,  247, 8757,  224,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 tensor(2))

In [22]:
train_load = DataLoader(train_set,batch_size=32,shuffle=True,pin_memory=True)
test_load  = DataLoader(test_set,batch_size=32,shuffle=False,pin_memory=True)

In [41]:
class MyRnn(nn.Module):
  def __init__(self,hidden_dim,embbed_dim,vocab_size,cats):
    super().__init__()

    self.embbedings =   nn.Embedding(vocab_size,embbed_dim)
    self.rnn  =    nn.RNN(embbed_dim,hidden_dim,batch_first=True)
    self.fc =   nn.Linear(hidden_dim,cats)
  def forward(self,x):
    emb = self.embbedings(x)
    _,h = self.rnn(emb)
    h_l = h.squeeze(0)
    return self.fc(h_l)

In [52]:
model = MyRnn(16,10,len(D),3)

In [53]:
torch.cuda.is_available()

True

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
model = model.to(device)

In [56]:
epoch = 500
lr = 0.01

In [57]:
loss_fun = nn.CrossEntropyLoss()
opt = torch.optim.Adagrad(model.parameters(),lr=0.03)

In [58]:
for i in range(epoch):
  batch_loss =0
  for batch,batch_label in train_load:
    batch = batch.to(device)
    batch_label = batch_label.to(device)
    logits = model(batch)
    loss = loss_fun(logits,batch_label)
    opt.zero_grad()
    loss.backward()
    opt.step()
    batch_loss += loss.item()
  print(f'Average batch loss in epoch{i} is {batch_loss/len(train_load)}')







Average batch loss in epoch0 is 1.0497511063303266
Average batch loss in epoch1 is 1.047859901700701
Average batch loss in epoch2 is 1.0477984939302718
Average batch loss in epoch3 is 1.0472133667128427
Average batch loss in epoch4 is 1.047420915876116
Average batch loss in epoch5 is 1.0467494320869446
Average batch loss in epoch6 is 1.0472900322505405
Average batch loss in epoch7 is 1.0470911724226815
Average batch loss in epoch8 is 1.0472050632749286
Average batch loss in epoch9 is 1.047230135372707
Average batch loss in epoch10 is 1.0468078967503138
Average batch loss in epoch11 is 1.0473878049850465
Average batch loss in epoch12 is 1.0466180290494647
Average batch loss in epoch13 is 1.0451447864941188
Average batch loss in epoch14 is 1.0451415484292166
Average batch loss in epoch15 is 1.0445384304864065
Average batch loss in epoch16 is 1.0449730243001665
Average batch loss in epoch17 is 1.03407566343035
Average batch loss in epoch18 is 1.0229172430719649
Average batch loss in epoch

In [65]:
model.fc.weight

Parameter containing:
tensor([[-2.8485e-01,  2.9835e-02,  4.4590e-01,  2.4673e-01, -1.8136e-01,
         -2.4069e-01, -3.3705e-01, -2.6289e-01, -5.6413e-01, -1.8893e-01,
          4.5167e-01,  2.1488e+00, -9.0250e-02, -2.4490e-01,  7.3689e-01,
          1.6344e+00],
        [ 9.3951e-05, -1.3747e-01, -9.2659e-01,  2.1089e-01, -5.6112e-01,
          4.7368e-01, -4.8952e-01, -3.5842e-02, -4.7588e-01,  7.3461e-02,
         -1.3295e+00, -2.4715e+00,  2.0336e-01, -2.3425e-01,  1.4892e+00,
         -1.3769e+00],
        [ 3.8911e-01,  7.2744e-02,  4.3962e-01, -5.4165e-01,  6.7231e-01,
         -4.7414e-01,  5.1284e-01,  1.0875e-01,  8.4943e-01,  2.1064e-01,
          5.3451e-01,  1.3609e-01, -8.0697e-03,  5.8111e-01, -2.4729e+00,
         -4.5529e-01]], device='cuda:0', requires_grad=True)

In [61]:
model.eval()

MyRnn(
  (embbedings): Embedding(13141, 10)
  (rnn): RNN(10, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=3, bias=True)
)

In [62]:
total = 0
corr =0
with torch.no_grad():
  for batch,batch_label in test_load:
    batch = batch.to(device)
    batch_label = batch_label.to(device)
    logits = model(batch)
    preds = torch.argmax(logits,dim=1)
    corr += (preds == batch_label).sum().item()
    total += batch.shape[0]
print(corr/total)



0.3335714285714286


In [67]:
preds = []
with torch.no_grad():
    for x, _ in test_load:
      x = x.to(device)
      preds.append(torch.argmax(model(x), dim=1))

preds = torch.cat(preds)
print(torch.bincount(preds))

tensor([546, 268, 586], device='cuda:0')
